In [26]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Conv1D,MaxPooling1D, Flatten

from keras.utils import multi_gpu_model
from keras import backend as K
from sklearn.cross_validation import train_test_split
import matplotlib
matplotlib.use('agg')
import pylab as plt

import pandas as pd
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import Normalizer
from keras.utils import np_utils
import pickle
import h5py


# KDD - and + 
X_testP_ = pd.read_pickle("./dataset/kdd_test_2labels.pkl")
X_trainN_ = pd.read_pickle("./dataset/kdd_train__2labels.pkl")
Y_trainN_ = X_trainN_.iloc[:, 122:].as_matrix()
Y_testP_ = X_testP_.iloc[:, 122:].as_matrix()
X_trainN_ = X_trainN_.as_matrix()
X_testP_ = X_testP_.as_matrix()
scaler1 = Normalizer().fit(X_trainN_[:,:122])
X_trainN_ = scaler1.transform(X_trainN_[:,:122])
scaler2 = Normalizer().fit(X_testP_[:,:122])
X_testP_ = scaler2.transform(X_testP_[:,:122])
X_trainN_ = np.reshape(X_trainN_, (X_trainN_.shape[0],X_trainN_.shape[1],1))
X_testP_ = np.reshape(X_testP_, (X_testP_.shape[0],X_testP_.shape[1],1))
print(Y_testP_.shape)
print(X_testP_.shape)
print(Y_trainN_.shape)
print(X_trainN_.shape)

(22544, 2)
(22544, 122, 1)
(25192, 2)
(25192, 122, 1)


In [27]:
cnn1d_1 = Sequential([Conv1D(64, 3, padding="same",input_shape=(122, 1)),
    Activation('relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(2),
    Activation('sigmoid'),
])
print(cnn1d_1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 122, 64)           256       
_________________________________________________________________
activation_4 (Activation)    (None, 122, 64)           0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 61, 64)            0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 3904)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               499840    
_________________________________________________________________
activation_5 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [28]:
#cnn1d_1 = multi_gpu_model(cnn1d_1, gpus=8)
cnn1d_1.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])
# train
start_time = time.time()
history = cnn1d_1.fit(X_trainN_, Y_trainN_, batch_size=64, validation_data=(X_testP_, Y_testP_) ,epochs=25)
end_time = time.time() 
print("Total time taken to train the Training model is", (end_time - start_time))
# serialize model to JSON
model_json = cnn1d_1.to_json()
with open("NP.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
cnn1d_1.save_weights("NP.h5")
print("Saved model to disk")
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig('Acc_ValaccuaracyNP.png')

Train on 25192 samples, validate on 22544 samples
Epoch 1/25
25192/25192 [==============================] - 7s 279us/step - loss: 0.2236 - acc: 0.9129 - val_loss: 0.8680 - val_acc: 0.6940
Epoch 2/25
25192/25192 [==============================] - 7s 268us/step - loss: 0.1499 - acc: 0.9446 - val_loss: 0.9232 - val_acc: 0.7334
Epoch 3/25
25192/25192 [==============================] - 7s 278us/step - loss: 0.1305 - acc: 0.9537 - val_loss: 0.9147 - val_acc: 0.7482
Epoch 4/25
25192/25192 [==============================] - 7s 281us/step - loss: 0.1222 - acc: 0.9575 - val_loss: 0.8461 - val_acc: 0.7624
Epoch 5/25
25192/25192 [==============================] - 7s 266us/step - loss: 0.1126 - acc: 0.9614 - val_loss: 0.9854 - val_acc: 0.7516
Epoch 6/25
25192/25192 [==============================] - 7s 272us/step - loss: 0.1084 - acc: 0.9617 - val_loss: 0.9442 - val_acc: 0.7653
Epoch 7/25
25192/25192 [==============================] - 7s 270us/step - loss: 0.1044 - acc: 0.9639 - val_loss: 1.0539 - 